# Imports

In [2]:
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.preprocessing import \
    create_windows_from_events, create_fixed_length_windows

from numpy import multiply

from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet
import tensorflow

2023-10-09 18:13:08.803490: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 18:13:08.803520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 18:13:08.803539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 18:13:08.809961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow not install, you could not use those pipelines


# Load, preprocess and get the data ready

In [3]:
subject_id = 3
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[subject_id])


In [3]:
################ Load ############################ 

subject_id = 3
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[subject_id])

################ Preprocess ############################ 
# Parameters for filtering
low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering

# Paramater for resampling
newfreq = 100

# Parameter for scaling
factor = 1e6

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda data: multiply(data, factor)),  # Convert from V to uV
    Preprocessor("resample", sfreq=newfreq), # Resampling
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor("set_eeg_reference", ref_channels="average", ch_type="eeg") # Common Average Reference
]

# Transform the data
preprocess(dataset, preprocessors, n_jobs = -1)

################ Create windows of data ############################ 

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=0,
    trial_stop_offset_samples=100,
    window_size_samples=400,
    window_stride_samples=100,
    preload=True,
    # mapping = {'left_hand': 0, 'right_hand': 1},
    picks = ['C3', 'Cz', 'C4']
)

/usr/local/lib/python3.11/dist-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')
2023-10-08 03:45:45.372657: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-08 03:45:45.372887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-08 03:45:45.372934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-08 03:45:45.384528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available

Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelines
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Tensorflow not install, you could not use those pipelin

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


# Split Train-Test

In [4]:
splitted = windows_dataset.split('session')
full_train_set = splitted['session_T']
evaluation_set = splitted['session_E']

In [5]:
# split by run, first 4 runs training, last 2 for validation
splitted_train = full_train_set.split([[0,1,2,3], [4,5]])

In [6]:
train_set = splitted_train['0']
valid_set = splitted_train['1']

In [7]:
train_set.description

,subject,session,run
0,3,session_T,run_0
1,3,session_T,run_1
2,3,session_T,run_2
3,3,session_T,run_3


In [8]:
valid_set.description

,subject,session,run
0,3,session_T,run_4
1,3,session_T,run_5


# Create model

In [9]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import EEGNetv4

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = False

seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = np.unique(train_set.datasets[0].y).size
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

F1, D = 8, 2

# Setting the length of the temporal kernel at half the sampling rate allows for
# capturing frequency information at 2Hz and above.
kernel_length = 64 # sfreq//4 = 62

model = EEGNetv4(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
    F1=8,
    D=2,
    F2=F1*D,
    kernel_length=kernel_length,
    drop_prob=0.5
)
model.to(device)

EEGNetv4(
  (ensuredims): Ensure4d()
  (dimshuffle): Expression(expression=_transpose_to_b_1_c_0) 
  (conv_temporal): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
  (bnorm_temporal): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (conv_spatial): Conv2dWithConstraint(8, 16, kernel_size=(3, 1), stride=(1, 1), groups=8, bias=False)
  (bnorm_1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (elu_1): Expression(expression=elu) 
  (pool_1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (drop_1): Dropout(p=0.5, inplace=False)
  (conv_separable_depth): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=16, bias=False)
  (conv_separable_point): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bnorm_2): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (elu_2): Expression(expression=elu) 
  (pool_2): AvgPool2d

# Training

In [10]:
train_set.get_metadata()["target"]

0     1
1     1
2     2
3     2
4     2
     ..
91    1
92    2
93    2
94    1
95    1
Name: target, Length: 384, dtype: int64

In [68]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier

batch_size = 32
n_epochs = 100

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    # optimizer__lr=lr,
    # optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=train_set.get_metadata()["target"], epochs=n_epochs);

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3594        1.4553            0.2604        1.3812  0.0100  0.5821
      2            0.3333        1.3408            0.2917        1.5203  0.0100  0.5499
      3            0.2708        1.1449            0.2552        2.5574  0.0100  0.5408
      4            0.3021        1.0383            0.2760        3.2029  0.0100  0.5487
      5            0.3698        0.8826            0.3125        2.5962  0.0100  0.5451
      6            0.3672        0.8449            0.3125        2.6102  0.0099  0.5504
      7            0.4792        0.8310            0.3646        2.6130  0.0099  0.5466
      8            0.6146        0.7601            0.4427        1.8218  0.0099  0.5449
      9            0.5677        0.7677            0.4219        2.2663  0.0098  0.5326
     10            0.5365       

In [188]:
windows_dataset.datasets[0].windows.to_data_frame().query("epoch == 0")

,time,condition,epoch,C3,Cz,C4
0,0.00,left_hand,0,-5.613643e+06,-1.652850e+06,1.735193e+06
1,0.01,left_hand,0,-4.486058e+06,-2.417474e+06,1.439939e+06
2,0.02,left_hand,0,-4.079097e+06,-1.397468e+06,-6.796606e+05
3,0.03,left_hand,0,-2.389204e+06,9.400325e+05,1.109374e+06
4,0.04,left_hand,0,2.592844e+06,-7.406544e+05,5.263186e+05
...,...,...,...,...,...,...
395,3.95,left_hand,0,2.981859e+06,1.028582e+06,-1.887287e+06
396,3.96,left_hand,0,3.703870e+06,1.350898e+06,-1.774051e+06
397,3.97,left_hand,0,2.458607e+06,1.277703e+05,5.419432e+05
398,3.98,left_hand,0,2.080754e+06,-3.015838e+06,-1.299250e+06


In [181]:
windows_dataset.datasets[0].windows.to_data_frame()

,time,condition,epoch,C3,Cz,C4
0,0.00,left_hand,0,-5.613643e+06,-1.652850e+06,1.735193e+06
1,0.01,left_hand,0,-4.486058e+06,-2.417474e+06,1.439939e+06
2,0.02,left_hand,0,-4.079097e+06,-1.397468e+06,-6.796606e+05
3,0.03,left_hand,0,-2.389204e+06,9.400325e+05,1.109374e+06
4,0.04,left_hand,0,2.592844e+06,-7.406544e+05,5.263186e+05
...,...,...,...,...,...,...
38395,3.95,tongue,95,-6.284606e+06,3.321265e+06,1.283233e+06
38396,3.96,tongue,95,2.011335e+06,4.248620e+06,-2.192824e+06
38397,3.97,tongue,95,7.710312e+06,1.393335e+06,-5.271722e+06
38398,3.98,tongue,95,6.351598e+06,1.022974e+06,-6.270476e+06


# DataFrame Format

In [194]:
windows_list = []
labels_list = []
for i in range(n_runs):
    windows_list.append(windows_dataset.datasets[i].windows.get_data())
    labels_list.append(windows_dataset.datasets[i].y)
    
stacked_tensor = np.concatenate(windows_list, axis=0)
stacked_labels = np.concatenate(labels_list, axis=0)